# <font color='Blue'>Clustering case study data using Hierarchical Clustering</font>

<b> Hierarchical clustering </b>

    Steps in executing Hierarchical clustering
       1. Decide the value of k
       2. Decide the distance type using parameter 'affinity' - “euclidean”, “l1”, “l2”, “manhattan”, “cosine”, or “precomputed”
       3. Decide the linkage type using parameter 'linkage' - “ward”, “complete”, “average”, “single”

# <font color='Blue'>Loading Libraries</font>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn

from sklearn.cluster import AgglomerativeClustering
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

## <font color='Blue'>1.0 Loading Data</font>

In [2]:
mktres = pd.read_csv( "mkt_res.csv" )
# Parking data in another dataframe
data = mktres
data.head(10)

,ID,Gender,Marital_Status,Work_Status,Education,Annual_Income,Age,Location,Purchasing_Decision_Maker,Purchasing_Location,Monthly_Electronics_Spend,Monthly_Household_Spend,Purchasing_Frequency,Technology_Adoption,Viewing_hours_day,Favorite_feature
0,1,male,married,professional,none,49,30,Florida,family,mass-consumer electronics,35,150,13,late,2,saving favorite shows to watch as a family
1,2,male,single,none,none,46,36,Alabama,single,mass-consumer electronics,35,163,26,late,10,saving favorite shows to watch as a family
2,3,male,married,professional,BA,58,66,Massachusetts,family,specialty stores,64,103,13,early,0,time shifting
3,4,male,married,none,PhD,51,78,New York,family,mass-consumer electronics,33,154,22,late,5,saving favorite shows to watch as a family
4,5,female,single,none,none,46,52,Montana,single,mass-consumer electronics,45,161,47,late,2,saving favorite shows to watch as a family
5,6,female,married,none,BA,31,72,New Jersey,single,retail,14,21,32,early,1,time shifting
6,7,male,married,professional,none,33,62,California,single,discount,18,40,41,early,0,cool gadget
7,8,male,married,none,none,29,30,New Hampshire,single,retail,23,75,9,early,1,schedule control
8,9,male,married,professional,none,57,60,Massachusetts,family,specialty stores,74,358,1,early,0,schedule control
9,10,female,married,professional,none,30,59,Idaho,family,discount,16,78,25,early,0,schedule control


## <font color='Blue'>1.1 Get the column names</font>

In [3]:
columns = list(data.columns) 
print(columns)
print("")

['ID', 'Gender', 'Marital_Status', 'Work_Status', 'Education', 'Annual_Income', 'Age', 'Location', 'Purchasing_Decision_Maker', 'Purchasing_Location', 'Monthly_Electronics_Spend', 'Monthly_Household_Spend', 'Purchasing_Frequency', 'Technology_Adoption', 'Viewing_hours_day', 'Favorite_feature']



## <font color='Blue'>1.2 Adding derived data</font>

In [4]:
data['Annual_Electronics_Spend'] = data['Monthly_Electronics_Spend']*12
data['Annual_Household_Spend']   = data['Monthly_Household_Spend']*12 
data['Electronic_Spend_Perc'] = (data['Annual_Electronics_Spend']/data['Annual_Household_Spend'])*100
data['Electronic_Spend_Perc'] = data['Electronic_Spend_Perc'].round(2)

# Number of raws and columns
print("#Rows and #Columns",data.shape)
print("")

#Rows and #Columns (1000, 19)



## <font color='Blue'>1.3 Drop columns not need for clustering</font>

In [5]:
data = data.drop(['ID','Location','Annual_Household_Spend','Monthly_Electronics_Spend','Monthly_Household_Spend'],axis=1)
print("#Rows and #Columns",data.shape)
print("")
columns = list(data.columns) 
print(columns)

#Rows and #Columns (1000, 14)

['Gender', 'Marital_Status', 'Work_Status', 'Education', 'Annual_Income', 'Age', 'Purchasing_Decision_Maker', 'Purchasing_Location', 'Purchasing_Frequency', 'Technology_Adoption', 'Viewing_hours_day', 'Favorite_feature', 'Annual_Electronics_Spend', 'Electronic_Spend_Perc']


## <font color='Blue'>1.4 Dummy Coding Variables</font>

In [6]:
dummy      = ['Gender', 'Marital_Status', 'Work_Status', 'Education','Purchasing_Decision_Maker','Purchasing_Location','Technology_Adoption','Favorite_feature']
dummydata  = pd.get_dummies(data, columns=dummy)
dummydata.head()

#Columns
print("#Rows and #Columns",dummydata.shape)
print("")
columns = list(dummydata.columns) 
print(columns)

#Rows and #Columns (1000, 30)

['Annual_Income', 'Age', 'Purchasing_Frequency', 'Viewing_hours_day', 'Annual_Electronics_Spend', 'Electronic_Spend_Perc', 'Gender_female', 'Gender_male', 'Marital_Status_married', 'Marital_Status_single', 'Work_Status_none', 'Work_Status_professional', 'Education_BA', 'Education_MA', 'Education_PhD', 'Education_none', 'Purchasing_Decision_Maker_family', 'Purchasing_Decision_Maker_single', 'Purchasing_Location_discount', 'Purchasing_Location_mass-consumer electronics', 'Purchasing_Location_retail', 'Purchasing_Location_specialty stores', 'Purchasing_Location_web (ebay)', 'Technology_Adoption_early', 'Technology_Adoption_late', 'Favorite_feature_cool gadget', 'Favorite_feature_programming/interactive features', 'Favorite_feature_saving favorite shows to watch as a family', 'Favorite_feature_schedule control', 'Favorite_feature_time shifting']


## <font color='Blue'>1.5 Examining Data</font>

In [7]:
dummydata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 30 columns):
 #   Column                                                       Non-Null Count  Dtype  
---  ------                                                       --------------  -----  
 0   Annual_Income                                                1000 non-null   int64  
 1   Age                                                          1000 non-null   int64  
 2   Purchasing_Frequency                                         1000 non-null   int64  
 3   Viewing_hours_day                                            1000 non-null   int64  
 4   Annual_Electronics_Spend                                     1000 non-null   int64  
 5   Electronic_Spend_Perc                                        1000 non-null   float64
 6   Gender_female                                                1000 non-null   uint8  
 7   Gender_male                                                  1000 non-null   ui

In [8]:
dummydata.head()

,Annual_Income,Age,Purchasing_Frequency,Viewing_hours_day,Annual_Electronics_Spend,Electronic_Spend_Perc,Gender_female,Gender_male,Marital_Status_married,Marital_Status_single,...,Purchasing_Location_retail,Purchasing_Location_specialty stores,Purchasing_Location_web (ebay),Technology_Adoption_early,Technology_Adoption_late,Favorite_feature_cool gadget,Favorite_feature_programming/interactive features,Favorite_feature_saving favorite shows to watch as a family,Favorite_feature_schedule control,Favorite_feature_time shifting
0,49,30,13,2,420,23.33,0,1,1,0,...,0,0,0,0,1,0,0,1,0,0
1,46,36,26,10,420,21.47,0,1,0,1,...,0,0,0,0,1,0,0,1,0,0
2,58,66,13,0,768,62.14,0,1,1,0,...,0,1,0,1,0,0,0,0,0,1
3,51,78,22,5,396,21.43,0,1,1,0,...,0,0,0,0,1,0,0,1,0,0
4,46,52,47,2,540,27.95,1,0,0,1,...,0,0,0,0,1,0,0,1,0,0


## <font color='Blue'>1.6 Normalizing Non-Categorical Variables</font>

In [9]:
scaler = StandardScaler()
dummydata[["Annual_Income"]]               = scaler.fit_transform(dummydata[["Annual_Income"]])
dummydata[["Age"]]                         = scaler.fit_transform(dummydata[["Age"]])
dummydata[["Purchasing_Frequency"]]        = scaler.fit_transform(dummydata[["Purchasing_Frequency"]])
dummydata[["Annual_Electronics_Spend"]]    = scaler.fit_transform(dummydata[["Annual_Electronics_Spend"]])
dummydata[["Electronic_Spend_Perc"]]       = scaler.fit_transform(dummydata[["Electronic_Spend_Perc"]])
dummydata[["Viewing_hours_day"]]           = scaler.fit_transform(dummydata[["Viewing_hours_day"]])
dummydata.head()

,Annual_Income,Age,Purchasing_Frequency,Viewing_hours_day,Annual_Electronics_Spend,Electronic_Spend_Perc,Gender_female,Gender_male,Marital_Status_married,Marital_Status_single,...,Purchasing_Location_retail,Purchasing_Location_specialty stores,Purchasing_Location_web (ebay),Technology_Adoption_early,Technology_Adoption_late,Favorite_feature_cool gadget,Favorite_feature_programming/interactive features,Favorite_feature_saving favorite shows to watch as a family,Favorite_feature_schedule control,Favorite_feature_time shifting
0,0.322666,-1.027302,-0.719876,-0.127200,0.236954,-0.915874,0,1,1,0,...,0,0,0,0,1,0,0,1,0,0
1,0.225798,-0.691290,0.217275,2.623078,0.236954,-1.026173,0,1,0,1,...,0,0,0,0,1,0,0,1,0,0
2,0.613268,0.988773,-0.719876,-0.814770,1.938706,1.385574,0,1,1,0,...,0,1,0,1,0,0,0,0,0,1
3,0.387244,1.660798,-0.071079,0.904154,0.119592,-1.028545,0,1,1,0,...,0,0,0,0,1,0,0,1,0,0
4,0.225798,0.204744,1.731134,-0.127200,0.823765,-0.641906,1,0,0,1,...,0,0,0,0,1,0,0,1,0,0


## <font color='Blue'>2.0 Generating Hierarchical Clusterining Solutions</font>

### <font color='Blue'>2.1 Generate 3 cluster and 4 cluster solution</font>

    Let's use different combinations of distance and linkage types
    We shall generate 3 and 4 cluster solution for different combinations of distance and linkage types
    Then observe cluster sizes and based on that retain the solution
    Based on the sizes, we could observe euclidean ward 3 cluster solution is okay

In [10]:
clusterid3 = AgglomerativeClustering(n_clusters=3,affinity='euclidean',linkage='ward').fit(dummydata).labels_
clusterid4 = AgglomerativeClustering(n_clusters=4,affinity='euclidean',linkage='ward').fit(dummydata).labels_
clusterid5 = AgglomerativeClustering(n_clusters=5,affinity='euclidean',linkage='ward').fit(dummydata).labels_
clusterid6 = AgglomerativeClustering(n_clusters=6,affinity='euclidean',linkage='ward').fit(dummydata).labels_

# You may try other combinations not listed here.

# Solution Sizes

# Euclidean Ward     3 , Euclidean Ward     4 
# Euclidean Complete 3 , Euclidean complete 4 
# Euclidean average  3 , Euclidean average  4 
# Euclidean single   3 , Euclidean single   3

# Manhattan Complete 3 , Manhattan complete 4
# Manhattan average  3 , Manhattan average  4
# Manhattan single   3 , Manhattan single   3

###  <font color='Blue'>2.2 Assign Cluster Labels</font>

In [11]:
data["clusterid3"] = clusterid3
data["clusterid4"] = clusterid4
data["clusterid5"] = clusterid5
data["clusterid6"] = clusterid6
cluster_size3 = data.groupby(['clusterid3']).size() 
cluster_size4 = data.groupby(['clusterid4']).size() 
cluster_size5 = data.groupby(['clusterid5']).size() 
cluster_size6 = data.groupby(['clusterid6']).size() 
print(cluster_size3)
print("")
print(cluster_size4)
print("")
print(cluster_size5)
print("")
print(cluster_size6)

clusterid3
0    162
1    638
2    200
dtype: int64

clusterid4
0    638
1    160
2    200
3      2
dtype: int64

clusterid5
0    504
1    134
2    200
3      2
4    160
dtype: int64

clusterid6
0    200
1    134
2    379
3      2
4    160
5    125
dtype: int64


###  <font color='Blue'>2.3 Performance Measure: Silhouette Score</font>

In [12]:
print("Silhouette Coefficient: %0.3f"% metrics.silhouette_score(dummydata, clusterid3))
print("Silhouette Coefficient: %0.3f"% metrics.silhouette_score(dummydata, clusterid4))
print("Silhouette Coefficient: %0.3f"% metrics.silhouette_score(dummydata, clusterid5))
print("Silhouette Coefficient: %0.3f"% metrics.silhouette_score(dummydata, clusterid6))
# Silhouette score between -1 and 1

Silhouette Coefficient: 0.232
Silhouette Coefficient: 0.224
Silhouette Coefficient: 0.227
Silhouette Coefficient: 0.183


###  <font color='Blue'>2.4 Performance Measure: Calinski-Harabasz Index</font>

In [13]:
print("Calinski-Harabasz index: %0.3f"% metrics.calinski_harabasz_score(dummydata, clusterid3))
print("Calinski-Harabasz index: %0.3f"% metrics.calinski_harabasz_score(dummydata, clusterid4))
print("Calinski-Harabasz index: %0.3f"% metrics.calinski_harabasz_score(dummydata, clusterid5))
print("Calinski-Harabasz index: %0.3f"% metrics.calinski_harabasz_score(dummydata, clusterid6))

Calinski-Harabasz index: 188.518
Calinski-Harabasz index: 181.949
Calinski-Harabasz index: 190.272
Calinski-Harabasz index: 170.192


<b> In the above solutions, in cluster 5 solution, we can omit the cluster with size 2 and treat it as a four cluster solution </b>

###  <font color='Blue'>3.0 Examining Chararcteristics</font>

In [14]:
values=['Annual_Income','Age','Purchasing_Frequency','Viewing_hours_day','Annual_Electronics_Spend','Electronic_Spend_Perc']
index =['clusterid5']
aggfunc={'Annual_Income': np.mean,
         'Age': np.mean,
         'Purchasing_Frequency':np.mean,
         'Viewing_hours_day':np.mean,
         'Annual_Electronics_Spend':np.mean,
         'Electronic_Spend_Perc':np.mean}
result = pd.pivot_table(data,values=values,
                             index =index,
                             aggfunc=aggfunc,
                             fill_value=0)
result['cluster_size'] = cluster_size5
result = result.round(2)
result

,Age,Annual_Electronics_Spend,Annual_Income,Electronic_Spend_Perc,Purchasing_Frequency,Viewing_hours_day,cluster_size
clusterid5,,,,,,,
0,51.99,204.12,30.02,38.83,24.75,1.03,504
1,24.30,407.28,30.87,40.98,23.91,3.03,134
2,53.50,496.86,48.10,38.29,29.94,6.32,200
3,80.00,720.00,685.00,45.72,8.00,1.50,2
4,50.15,708.00,54.71,37.27,8.16,1.10,160


###  <font color='Blue'>3.1 Examining Chararcteristics - Cont'd</font>

In [15]:
dummydata['clusterid5'] = clusterid5
values=['Gender_female','Gender_male','Marital_Status_married','Marital_Status_single','Work_Status_none',
        'Work_Status_professional']
index =['clusterid5']
aggfunc={'Gender_female': np.mean,
         'Gender_male': np.mean,
         'Marital_Status_married':np.mean,
         'Marital_Status_single':np.mean,
         'Work_Status_none':np.mean,
         'Work_Status_professional':np.mean}
result = pd.pivot_table(dummydata,values=values,
                             index =index,
                             aggfunc=aggfunc,
                             fill_value=0)
result['cluster_size'] = cluster_size5
result = result.round(2)
result

,Gender_female,Gender_male,Marital_Status_married,Marital_Status_single,Work_Status_none,Work_Status_professional,cluster_size
clusterid5,,,,,,,
0,0.48,0.52,0.80,0.20,0.5,0.5,504
1,0.54,0.46,0.06,0.94,0.0,1.0,134
2,0.53,0.47,0.90,0.10,0.5,0.5,200
3,0.00,1.00,1.00,0.00,0.0,1.0,2
4,0.28,0.72,0.79,0.21,0.0,1.0,160


###  <font color='Blue'>3.2 Examining Chararcteristics - Cont'd</font>

In [16]:
dummydata['clusterid5'] = clusterid5
values=['Education_BA','Education_MA','Education_PhD','Education_none','Purchasing_Decision_Maker_family',
        'Purchasing_Decision_Maker_single']
index =['clusterid5']
aggfunc={'Education_BA': np.mean,
         'Education_MA': np.mean,
         'Education_PhD':np.mean,
         'Education_none':np.mean,
         'Purchasing_Decision_Maker_family':np.mean,
         'Purchasing_Decision_Maker_single':np.mean}
result = pd.pivot_table(dummydata,values=values,
                             index =index,
                             aggfunc=aggfunc,
                             fill_value=0)
result['cluster_size'] = cluster_size5
result = result.round(2)
result

,Education_BA,Education_MA,Education_PhD,Education_none,Purchasing_Decision_Maker_family,Purchasing_Decision_Maker_single,cluster_size
clusterid5,,,,,,,
0,0.15,0.03,0.02,0.79,0.50,0.50,504
1,0.25,0.25,0.26,0.23,0.00,1.00,134
2,0.25,0.25,0.25,0.25,0.90,0.10,200
3,0.50,0.00,0.00,0.50,1.00,0.00,2
4,0.61,0.15,0.09,0.15,0.79,0.21,160


###  <font color='Blue'>3.3 Examining Chararcteristics - Cont'd</font>

In [17]:
dummydata['clusterid5'] = clusterid5
values=['Technology_Adoption_early','Technology_Adoption_late']
index =['clusterid5']
aggfunc={'Technology_Adoption_early': np.mean,
         'Technology_Adoption_late': np.mean}
result = pd.pivot_table(dummydata,values=values,
                             index =index,
                             aggfunc=aggfunc,
                             fill_value=0)
result['cluster_size'] = cluster_size5
result = result.round(2)
result

,Technology_Adoption_early,Technology_Adoption_late,cluster_size
clusterid5,,,
0,1,0,504
1,1,0,134
2,0,1,200
3,1,0,2
4,1,0,160


###  <font color='Blue'>3.4 Examining Chararcteristics - Cont'd</font>

In [18]:
dummydata['clusterid5'] = clusterid5
values=['Favorite_feature_cool gadget','Favorite_feature_programming/interactive features','Favorite_feature_saving favorite shows to watch as a family',
        'Favorite_feature_schedule control','Favorite_feature_time shifting']
index =['clusterid5']
aggfunc={'Favorite_feature_cool gadget': np.mean,
         'Favorite_feature_programming/interactive features': np.mean,
         'Favorite_feature_saving favorite shows to watch as a family':np.mean,
         'Favorite_feature_schedule control':np.mean,
         'Favorite_feature_time shifting':np.mean}
result = pd.pivot_table(dummydata,values=values,
                             index =index,
                             aggfunc=aggfunc,
                             fill_value=0)
result['cluster_size'] = cluster_size5
result = result.round(2)
result

,Favorite_feature_cool gadget,Favorite_feature_programming/interactive features,Favorite_feature_saving favorite shows to watch as a family,Favorite_feature_schedule control,Favorite_feature_time shifting,cluster_size
clusterid5,,,,,,
0,0.34,0.00,0,0.33,0.33,504
1,0.02,0.96,0,0.01,0.01,134
2,0.00,0.00,1,0.00,0.00,200
3,0.50,0.00,0,0.50,0.00,2
4,0.33,0.00,0,0.33,0.34,160
